In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [67.0 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://arch

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
#!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-01 01:43:36--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.08MB/s    in 0.2s    

2021-09-01 01:43:37 (6.08 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10293599|R14LVB34Z2Z53I|B000YMOQZY|     262935067|Knockin' On Heave...|Digital_Music_Pur...|          5|            0|          0|   N|                Y|           favorites|Great  rendition....| 2015-08-31|
|         US|    6087195|R2M0R9CWPC82LT|B00ISAEC80|     452318038|            Flawless|Digital_Music_Pur

### Create vine DataFrames

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14LVB34Z2Z53I|          5|            0|          0|   N|                Y|
|R2M0R9CWPC82LT|          5|            0|          0|   N|                Y|
| RQNQI2R3BM52A|          4|            0|          0|   N|                Y|
| RTUIX1135HUAB|          5|            0|          0|   N|                Y|
| RE3QSAT8TL010|          4|            0|          0|   N|                Y|
|R14TN65IY0NUOD|          5|            0|          0|   N|                Y|
|R3LIF8MCNH1ODF|          5|            0|          0|   N|                Y|
|R33X0DGA4PPQ3L|          5|            3|          3|   N|                N|
|R2DJJP7DQRT1ZW|          5|            0|          1|   N|                Y|
|R2A8XDXW5XOT4H|          5|            0|          0|   N|     

In [6]:
# Filter the DF where total_votes equal or greater than 20
votes_df = vine_df.filter("total_votes >=20")
votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1S0N56H5B9SUX|          5|           30|         30|   N|                Y|
| RCDI7Z8VCZJI9|          4|           40|         43|   N|                N|
|R3F4225VTUO2WE|          1|            8|         33|   N|                N|
|  R3ADPNURCCTK|          1|            1|         39|   N|                N|
|R2LWBYC8TEUVF2|          5|           50|         53|   N|                Y|
|R23E4EIJXID5IC|          5|           43|         43|   N|                N|
|R1WFSWDT8L5RSJ|          4|           23|         26|   N|                N|
|R37LMHN02KOER1|          5|           50|         50|   N|                N|
|R297QI4VU3LMY6|          4|           41|         50|   N|                N|
| RGNU1L1XFX33B|          3|           34|         35|   N|     

In [7]:
# Filter where Ratio between Helpful votes and total votes equal or greater than 50%
vote_ratio_df = votes_df.filter("helpful_votes/total_votes >= 0.5")
vote_ratio_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1S0N56H5B9SUX|          5|           30|         30|   N|                Y|
| RCDI7Z8VCZJI9|          4|           40|         43|   N|                N|
|R2LWBYC8TEUVF2|          5|           50|         53|   N|                Y|
|R23E4EIJXID5IC|          5|           43|         43|   N|                N|
|R1WFSWDT8L5RSJ|          4|           23|         26|   N|                N|
|R37LMHN02KOER1|          5|           50|         50|   N|                N|
|R297QI4VU3LMY6|          4|           41|         50|   N|                N|
| RGNU1L1XFX33B|          3|           34|         35|   N|                N|
|R2NPE00F48X0ET|          5|           22|         27|   N|                N|
| RSPQFPJ4ISKS3|          5|           32|         35|   N|     

In [15]:
# Filter for vine votes

vine_yes_votes_df = vote_ratio_df.filter(vote_ratio_df.vine == 'Y')
vine_yes_votes_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [16]:
# Filter for non vine votes
vine_no_votes_df = vote_ratio_df.filter(vote_ratio_df.vine == 'N')
vine_no_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1S0N56H5B9SUX|          5|           30|         30|   N|                Y|
| RCDI7Z8VCZJI9|          4|           40|         43|   N|                N|
|R2LWBYC8TEUVF2|          5|           50|         53|   N|                Y|
|R23E4EIJXID5IC|          5|           43|         43|   N|                N|
|R1WFSWDT8L5RSJ|          4|           23|         26|   N|                N|
|R37LMHN02KOER1|          5|           50|         50|   N|                N|
|R297QI4VU3LMY6|          4|           41|         50|   N|                N|
| RGNU1L1XFX33B|          3|           34|         35|   N|                N|
|R2NPE00F48X0ET|          5|           22|         27|   N|                N|
| RSPQFPJ4ISKS3|          5|           32|         35|   N|     

In [36]:
# 5 Star votes count by Vine paid (Y) and unpaid (N)

import pyspark.sql.functions as func

star_df = votes_df.filter("star_rating = 5")
star_five_df = star_df.groupby("vine").agg(func.count("star_rating")).withColumnRenamed("count(star_rating)", "5 Star Count")
star_five_df.show()

+----+------------+
|vine|5 Star Count|
+----+------------+
|   N|        2658|
+----+------------+



In [35]:
# Total star review count
total_star_df = votes_df.groupby("vine").agg(func.count("star_rating")).withColumnRenamed("count(star_rating)", "Star Count")
total_star_df.show()

+----+----------+
|vine|Star Count|
+----+----------+
|   N|      7156|
+----+----------+



In [38]:
join_star_df = total_star_df.join(star_five_df, on="vine",how="inner")
join_star_df.show()

+----+----------+------------+
|vine|Star Count|5 Star Count|
+----+----------+------------+
|   N|      7156|        2658|
+----+----------+------------+



In [43]:
# calculate percentage of 5 star review for paid and non paid reviews
pct_star_df = join_star_df.withColumn("5 Star Percentage", join_star_df["5 Star Count"]/join_star_df["Star Count"]*100)
pct_star_df.show()

+----+----------+------------+-----------------+
|vine|Star Count|5 Star Count|5 Star Percentage|
+----+----------+------------+-----------------+
|   N|      7156|        2658|37.14365567356065|
+----+----------+------------+-----------------+

